In [1]:
import os
import cv2
import pandas as pd
import numpy as np
from tensorflow import keras
from tensorflow.keras.models import Model
from tensorflow.keras.layers import (Flatten, Dense, Input, 
                                     Dropout, BatchNormalization,
                                     Conv2D, MaxPooling2D, 
                                     GlobalMaxPooling2D, GlobalAveragePooling2D,)

from tensorflow.keras import backend as K
from sklearn.model_selection import train_test_split

In [2]:
def cv_show(img, name='image'):
    cv2.imshow(name, img)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

In [54]:
def preprocessing_img(img):
    # 先padding再裁切
    
    out_w = 128
    out_h = 128
    top_size = int((128 - img.shape[0]) / 2)
    bottom_size = int((128 - img.shape[0]) / 2)
    dif_1 = 128 - (top_size + bottom_size + img.shape[0])
    top_size += dif_1
    left_size = int((128 - img.shape[1]) / 2)
    right_size = int((128 - img.shape[1]) / 2)
    dif_2 = 128 - (left_size + right_size + img.shape[1])
    left_size += dif_2    
    padding_v = tuple(img[-1, -1, :])
    if img.shape[1] <= 128:
        padding = cv2.copyMakeBorder(img, 
                                  top_size, bottom_size, left_size, right_size, 
                                  borderType=cv2.BORDER_CONSTANT, 
                                  value=(int(padding_v[0]), int(padding_v[1]), int(padding_v[2])))
    else:
        padding = cv2.copyMakeBorder(img, 
                          top_size, bottom_size, 0, 0, 
                          borderType=cv2.BORDER_CONSTANT, 
                          value=(int(padding_v[0]), int(padding_v[1]), int(padding_v[2])))
        padding = cv2.resize(padding, (128, 128))


    # 灰階 二值化
    gray = cv2.cvtColor(padding, cv2.COLOR_BGR2GRAY)
    ret, thresh = cv2.threshold(gray, 100, 255, cv2.THRESH_OTSU)
    # 黑白轉換
    thresh = 255 - thresh
    # 閉操作
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (9, 9))
    closed = cv2.morphologyEx(thresh, cv2.MORPH_CLOSE, kernel)
    # 腐蝕 & 膨脹
    closed = cv2.erode(closed, None, iterations=1)
    closed = cv2.dilate(closed, None, iterations=1)
    # 輪廓檢測
    contours, hierarchy = cv2.findContours(closed, cv2.RETR_TREE, cv2.CHAIN_APPROX_NONE)
    if len(contours) == 0:
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        ret, thresh = cv2.threshold(gray, 220, 255, cv2.THRESH_BINARY)
        thresh = 255 - thresh

    else:    
        # 取出最大的
        c = sorted(contours, key=cv2.contourArea, reverse=True)[0]
        if (cv2.contourArea(c) > 1000) and (len(contours) > 1):
            # 閉操作
            kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (5, 5))
            closed = cv2.morphologyEx(thresh, cv2.MORPH_CLOSE, kernel)
            # 腐蝕 & 膨脹
            closed = cv2.erode(closed, None, iterations=1)
            closed = cv2.dilate(closed, None, iterations=1)
            # 輪廓檢測
            contours, hierarchy = cv2.findContours(closed, cv2.RETR_TREE, cv2.CHAIN_APPROX_NONE)
            c = sorted(contours, key=cv2.contourArea, reverse=True)[0]
            
        rect = cv2.minAreaRect(c)
        box = np.int0(cv2.boxPoints(rect))
        # 裁切
        Xs = [i[0] for i in box]
        Ys = [i[1] for i in box]
        x1 = min(Xs)
        x2 = max(Xs)
        y1 = min(Ys)
        y2 = max(Ys)
        hight = y2 - y1
        width = x2 - x1
#         padding = padding[y1: y1+hight, x1: x1+width]
        thresh = thresh[y1: y1+hight, x1: x1+width]
        
    # reshape
    img2 = cv2.resize(thresh, (64, 64))
    img2 = cv2.cvtColor(img2, cv2.COLOR_GRAY2BGR)
     
    return img2

In [55]:
def load_data(path):
    
    fold_list = os.listdir(train_dir)
    trainX_list = []
    trainY_list = []
    testX_list = []
    testY_list = []

    for f in fold_list:
        label = int(f)
        img_list = os.listdir(f'{train_dir}{f}')
        tmp_datasetX = []
        tmp_datasetY = []
        for img_p in img_list:
            try:
                img = cv2.imread(f'{train_dir}{f}/{img_p}')
                img = preprocessing_img(img)
                label = int(f)
                tmp_datasetX.append(img)
                tmp_datasetY.append(label)
            except Exception as e:
                print(f, img_p)
                print(e)
        
        trainX, testX, trainY, testY = train_test_split(tmp_datasetX, tmp_datasetY, test_size=0.2, random_state=42)
        trainX_list.extend(trainX)
        trainY_list.extend(trainY)
        testX_list.extend(testX)
        testY_list.extend(testY)
    trainX_ary = np.array(trainX_list)
    trainY_ary = np.array(trainY_list)
    testX_ary = np.array(testX_list)
    testY_ary = np.array(testY_list)
    print(f'trainX: {trainX_ary.shape}')
    print(f'trainY: {trainY_ary.shape}')
    print(f'testX: {testX_ary.shape}')
    print(f'testY: {testY_ary.shape}')
        
    return trainX_ary, testX_ary, trainY_ary, testY_ary

In [56]:
train_dir = r'./data-clean/'
df = pd.read_csv('word_idx_df.csv', converters={'idx':str})

In [57]:
trainX, testX, trainY, testY = load_data(train_dir)


22 8844.jpg
OpenCV(4.5.1) C:\Users\appveyor\AppData\Local\Temp\1\pip-req-build-oduouqig\opencv\modules\imgproc\src\resize.cpp:4051: error: (-215:Assertion failed) !ssize.empty() in function 'cv::resize'

595 58299.jpg
OpenCV(4.5.1) C:\Users\appveyor\AppData\Local\Temp\1\pip-req-build-oduouqig\opencv\modules\imgproc\src\resize.cpp:4051: error: (-215:Assertion failed) !ssize.empty() in function 'cv::resize'

633 15394.jpg
OpenCV(4.5.1) C:\Users\appveyor\AppData\Local\Temp\1\pip-req-build-oduouqig\opencv\modules\imgproc\src\resize.cpp:4051: error: (-215:Assertion failed) !ssize.empty() in function 'cv::resize'

691 22184.jpg
OpenCV(4.5.1) C:\Users\appveyor\AppData\Local\Temp\1\pip-req-build-oduouqig\opencv\modules\imgproc\src\resize.cpp:4051: error: (-215:Assertion failed) !ssize.empty() in function 'cv::resize'

691 42847.jpg
OpenCV(4.5.1) C:\Users\appveyor\AppData\Local\Temp\1\pip-req-build-oduouqig\opencv\modules\imgproc\src\resize.cpp:4051: error: (-215:Assertion failed) !ssize.empty(

In [58]:
trainY = keras.utils.to_categorical(trainY, num_classes=800)
testY = keras.utils.to_categorical(testY, num_classes=800)

In [59]:
print(trainX.shape)
print(testX.shape)
print(trainY.shape)
print(testY.shape)

(47925, 64, 64, 3)
(12390, 64, 64, 3)
(47925, 800)
(12390, 800)


In [60]:
import gc
gc.collect()

5842

In [61]:
input_tensor = Input(shape=(64, 64, 3))
# input_tensor = Input(shape=(32, 32, 3))

model = keras.applications.Xception(include_top=False, 
                                    weights='imagenet',
                                    input_tensor=input_tensor,
                                    pooling=None,
                                    classes=800)
x = model.output
x = Flatten()(x)
x = Dense(1024, activation='relu')(x)
x = BatchNormalization()(x)
# x = Dense(1024, activation='relu')(x)
# x = BatchNormalization()(x)
predictions = Dense(800, activation='softmax')(x)
model = Model(inputs=model.input, outputs=predictions)

In [62]:
len(model.layers)

136

In [63]:
for layer in model.layers[:76]:
    layer.trainable = False
for layer in model.layers[76:]:
    layer.trainable = True

In [64]:
model.layers[76].name

'block9_sepconv1_act'

In [65]:
model.summary()

Model: "model_3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_4 (InputLayer)            [(None, 64, 64, 3)]  0                                            
__________________________________________________________________________________________________
block1_conv1 (Conv2D)           (None, 31, 31, 32)   864         input_4[0][0]                    
__________________________________________________________________________________________________
block1_conv1_bn (BatchNormaliza (None, 31, 31, 32)   128         block1_conv1[0][0]               
__________________________________________________________________________________________________
block1_conv1_act (Activation)   (None, 31, 31, 32)   0           block1_conv1_bn[0][0]            
____________________________________________________________________________________________

In [66]:
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from tensorflow.keras import optimizers, callbacks

In [67]:
opt = optimizers.Adam(lr=0.001) 
model.compile(loss='categorical_crossentropy', 
              optimizer=opt,
              metrics=['accuracy'])

callbacks = [
    EarlyStopping(monitor='val_loss', patience=10, verbose=0),
    ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=7, verbose=1, min_delta=1e-4, mode='min')
    ]

In [68]:
model.fit(trainX, trainY, batch_size=32, epochs=50, verbose=2,
          validation_data=(testX, testY), callbacks=callbacks ) 

Epoch 1/50
1498/1498 - 59s - loss: 6.7132 - accuracy: 0.0075 - val_loss: 6.3427 - val_accuracy: 0.0186 - lr: 0.0010
Epoch 2/50
1498/1498 - 59s - loss: 5.2128 - accuracy: 0.0586 - val_loss: 4.7741 - val_accuracy: 0.1049 - lr: 0.0010
Epoch 3/50
1498/1498 - 59s - loss: 4.2331 - accuracy: 0.1593 - val_loss: 4.1593 - val_accuracy: 0.1808 - lr: 0.0010
Epoch 4/50
1498/1498 - 59s - loss: 3.4994 - accuracy: 0.2674 - val_loss: 3.5831 - val_accuracy: 0.2786 - lr: 0.0010
Epoch 5/50
1498/1498 - 60s - loss: 2.9703 - accuracy: 0.3552 - val_loss: 3.3191 - val_accuracy: 0.3254 - lr: 0.0010
Epoch 6/50
1498/1498 - 60s - loss: 2.5423 - accuracy: 0.4330 - val_loss: 3.1808 - val_accuracy: 0.3491 - lr: 0.0010
Epoch 7/50
1498/1498 - 60s - loss: 2.1702 - accuracy: 0.5026 - val_loss: 3.3704 - val_accuracy: 0.3484 - lr: 0.0010
Epoch 8/50
1498/1498 - 60s - loss: 1.8364 - accuracy: 0.5660 - val_loss: 3.1680 - val_accuracy: 0.3945 - lr: 0.0010
Epoch 9/50
1498/1498 - 60s - loss: 1.5517 - accuracy: 0.6271 - val_loss: